In [1]:
import pandas as pd 

In [3]:
!pip install latexify-py

ERROR: Ignored the following versions that require a different python version: 0.0.4 Requires-Python >=3.6, <3.9; 0.0.5 Requires-Python >=3.6, <3.9; 0.0.6 Requires-Python >=3.6, <3.9; 0.0.7 Requires-Python >=3.6, <3.9; 0.1.0 Requires-Python <3.11,>=3.7; 0.1.1 Requires-Python <3.11,>=3.7; 0.2.0 Requires-Python <3.11,>=3.7; 0.2.0b1 Requires-Python <3.11,>=3.7; 0.2.0b2 Requires-Python <3.11,>=3.7; 0.3.0b1 Requires-Python <3.12,>=3.7; 0.3.1 Requires-Python <3.12,>=3.7; 0.4.0 Requires-Python <3.12,>=3.7; 0.4.1 Requires-Python <3.12,>=3.7; 0.4.2 Requires-Python <3.12,>=3.7; 0.4.3.post1 Requires-Python <3.12,>=3.7
ERROR: Could not find a version that satisfies the requirement latexify-py (from versions: none)
ERROR: No matching distribution found for latexify-py

[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
from IPython.display import display, Math, Latex

# Inline LaTeX (will be embedded within the text)
display(Latex(r'Let $f(x) = x^2 + 2x + 1$. The derivative $f\'(x) = 2x + 2$.'))

<IPython.core.display.Latex object>

In [10]:
import json
import pandas as pd

# Load the JSON schema
with open('C:/Users/baljeet/Downloads/schema.json', 'r') as file:
    schema = json.load(file)

# Create a dictionary to hold DataFrames for each section
dataframes = {}

# Process each section and convert to DataFrame
for section, fields in schema.items():
    # Convert list of dictionaries to DataFrame
    df = pd.DataFrame(fields)
    dataframes[section] = df
    # print(fields)

# Write each section DataFrame to a separate sheet in an Excel file
excel_path = 'C:/Users/baljeet/Downloads/schema_output.xlsx'
with pd.ExcelWriter(excel_path) as writer:
    for section, df in dataframes.items():
        df.to_excel(writer, sheet_name=section, index=False)

print(f"Schema has been saved to {excel_path}")
# dataframes

Schema has been saved to C:/Users/baljeet/Downloads/schema_output.xlsx


In [8]:
!pip install openpyxl

   ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
   ---------------------------------------- 250.9/250.9 kB 7.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 24.1.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
import numpy as np
import plotly.graph_objects as go

# Generate dummy data with larger number of points
np.random.seed(0)
x = np.random.randint(1, 11, 1000)
decision_boundary_x = np.linspace(0, 10, 100)
decision_boundary_y = np.piecewise(
    decision_boundary_x,
    [
        (decision_boundary_x >= 0) & (decision_boundary_x < 1),
        (decision_boundary_x >= 1) & (decision_boundary_x < 2),
        (decision_boundary_x >= 2) & (decision_boundary_x < 4),
        (decision_boundary_x == 4),
        (decision_boundary_x > 4) & (decision_boundary_x < 6),
        (decision_boundary_x >= 6)
    ],
    [
        lambda x: 80000,
        lambda x: 80000 + (10000 * (x - 1)) + np.random.uniform(-2000, 2000, size=len(x)),
        lambda x: 180000,
        lambda x: 80000,
        lambda x: 80000 - (17500 * (x - 4)) + np.random.uniform(-2000, 2000, size=len(x)),
        lambda x: 45000
    ]
)

# Generate y values for inliers and outliers
y_inliers = np.array([decision_boundary_y[np.searchsorted(decision_boundary_x, xi)] - np.random.uniform(0, 40000) for xi in x[:800]])
y_outliers = np.array([decision_boundary_y[np.searchsorted(decision_boundary_x, xi)] + np.random.uniform(0, 40000) for xi in x[800:]])

# Combine inliers and outliers
x_inliers = x[:800]
x_outliers = x[800:]

# Create Plotly scatter plot
fig = go.Figure()

# Fill area below the boundary with darker orange
fig.add_trace(go.Scatter(
    x=np.concatenate([decision_boundary_x, decision_boundary_x[::-1]]),
    y=np.concatenate([decision_boundary_y, np.zeros_like(decision_boundary_x)]),
    fill='toself',
    fillcolor='rgba(255, 165, 0, 0.8)',  # Darker orange color with transparency
    line=dict(color='rgba(255, 165, 0, 0)'),
    showlegend=False
))

# Add decision boundary with hover text
fig.add_trace(go.Scatter(
    x=decision_boundary_x, y=decision_boundary_y, mode='lines', name='Decision Boundary',
    line=dict(color='green'),
    text=[f'Decision Boundary<br>x: {x:.2f}<br>y: {y:.2f}' for x, y in zip(decision_boundary_x, decision_boundary_y)],
    hoverinfo='text'
))

# Add inliers with hover text
fig.add_trace(go.Scatter(
    x=x_inliers, y=y_inliers, mode='markers', name='Inliers',
    marker=dict(color='blue'),
    text=[f'Inlier<br>x: {x}<br>y: {y}' for x, y in zip(x_inliers, y_inliers)],
    hoverinfo='text'
))

# Add outliers with hover text
fig.add_trace(go.Scatter(
    x=x_outliers, y=y_outliers, mode='markers', name='Outliers',
    marker=dict(color='red'),
    text=[f'Outlier<br>x: {x}<br>y: {y}' for x, y in zip(x_outliers, y_outliers)],
    hoverinfo='text'
))

# Update layout
fig.update_layout(
    title={
        'text': "Anomaly Detection - Isolation Forest",
        'x': 0.5,  # Center align title horizontally
        'xanchor': 'center'  # Center align title horizontally
    },
    xaxis_title="5-day count of transactions",
    yaxis_title="5-day sum of transactions",
    legend_title="Legend",
    template="plotly_dark",  # Dark grey background template
    xaxis=dict(tickmode='array', tickvals=np.arange(1, 11)),
    yaxis=dict(tickformat=",", tickprefix="$"),
    showlegend=True,
    height=800,  # Increased plot height
)

fig.show()
